<a href="https://colab.research.google.com/github/OneFineStarstuff/Onefinebot/blob/main/Physics_Informed_Neural_Networks_(PINNs)_for_Solving_Differential_Equations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Define the physics-informed neural network (PINN) model
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        return self.fc3(x)

# Define a differential equation, e.g., u''(x) = -u(x) (simple harmonic oscillator)
def physics_loss(x, model):
    x.requires_grad = True
    u = model(x)
    u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x), create_graph=True)[0]
    return ((u_xx + u)**2).mean()

# Generate training data for the differential equation
x_train = torch.linspace(0, 2 * np.pi, 100).reshape(-1, 1)

# Instantiate and train the model
model = PINN()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
n_epochs = 1000
for epoch in range(n_epochs):
    optimizer.zero_grad()
    loss = physics_loss(x_train, model)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Plot the solution
x_test = torch.linspace(0, 2 * np.pi, 100).reshape(-1, 1)
y_pred = model(x_test).detach().numpy()

plt.plot(x_test, y_pred, label="PINN Solution")
plt.xlabel("x")
plt.ylabel("u(x)")
plt.title("Physics-Informed Neural Network Solution")
plt.legend()
plt.show()